# 유저 분류
예측 변수 Segmentation, test.csv에 대해 ID별로 Segmentation의 클래스를 예측해서 저장후 제출, 

제출 데이터 컬럼은 ID와 Segmentation 두개만 존재해야함. 평가지표는 macro f1 score

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score

In [2]:
# sklearn.__all__

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e4_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e4_p2_test.csv')

display(train.head())
test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,464357,Male,No,40,Yes,Artist,7.0,Low,1.0,Cat_6,A
1,459624,Male,No,18,No,Healthcare,NaN,Low,5.0,Cat_4,D
2,462672,Male,No,25,No,Healthcare,7.0,Low,4.0,Cat_4,D
3,463360,Female,Yes,46,Yes,Artist,2.0,Low,1.0,Cat_6,B
4,462420,Male,No,27,Yes,Healthcare,1.0,Low,3.0,Cat_6,C


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,460406,Male,Yes,36,Yes,Healthcare,3.0,Low,2.0,Cat_6
1,466890,Male,Yes,47,Yes,Artist,0.0,Average,6.0,Cat_7
2,466145,Male,Yes,56,No,Entertainment,1.0,Average,3.0,Cat_6
3,465805,Male,Yes,50,Yes,Engineer,NaN,Low,1.0,Cat_3
4,466137,Female,NaN,31,No,Healthcare,0.0,Low,4.0,Cat_6


## EDA

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4881 entries, 0 to 4880
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               4881 non-null   int64  
 1   Gender           4881 non-null   object 
 2   Ever_Married     4785 non-null   object 
 3   Age              4881 non-null   int64  
 4   Graduated        4830 non-null   object 
 5   Profession       4817 non-null   object 
 6   Work_Experience  4368 non-null   float64
 7   Spending_Score   4881 non-null   object 
 8   Family_Size      4689 non-null   float64
 9   Var_1            4831 non-null   object 
 10  Segmentation     4881 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 419.6+ KB


## 결측치 처리

In [5]:
train.isnull().sum() 

ID                   0
Gender               0
Ever_Married        96
Age                  0
Graduated           51
Profession          64
Work_Experience    513
Spending_Score       0
Family_Size        192
Var_1               50
Segmentation         0
dtype: int64

In [6]:
test.isnull().sum() 

ID                   0
Gender               0
Ever_Married        22
Age                  0
Graduated           16
Profession          23
Work_Experience    142
Spending_Score       0
Family_Size         68
Var_1               12
dtype: int64

In [7]:
train.Ever_Married[train.Ever_Married.isnull()] = 'Not'
test.Ever_Married[test.Ever_Married.isnull()] = 'Not'

C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\741787156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Ever_Married[train.Ever_Married.isnull()] = 'Not'
C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\741787156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Ever_Married[test.Ever_Married.isnull()] = 'Not'


In [8]:
train.Graduated[train.Graduated.isnull()] = 'Not'
test.Graduated[test.Graduated.isnull()] = 'Not'

C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\1381527392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Graduated[train.Graduated.isnull()] = 'Not'
C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\1381527392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Graduated[test.Graduated.isnull()] = 'Not'


In [9]:
train.Profession[train.Profession.isnull()] = 'Not'
test.Profession[test.Profession.isnull()] = 'Not'

C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\2323427112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Profession[train.Profession.isnull()] = 'Not'
C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\2323427112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Profession[test.Profession.isnull()] = 'Not'


In [10]:
train.Work_Experience[train.Work_Experience.isnull()] = 0.5
test.Work_Experience[test.Work_Experience.isnull()] = 0.5

C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\586143673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Work_Experience[train.Work_Experience.isnull()] = 0.5
C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\586143673.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Work_Experience[test.Work_Experience.isnull()] = 0.5


In [11]:
train.Family_Size[train.Family_Size.isnull()] = 1.5
test.Family_Size[test.Family_Size.isnull()] = 1.5

C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\2908951001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Family_Size[train.Family_Size.isnull()] = 1.5
C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\2908951001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Family_Size[test.Family_Size.isnull()] = 1.5


In [14]:
train.Var_1[train.Var_1.isnull()] = 'Cat_6'
test.Var_1[test.Var_1.isnull()] = 'Cat_6'

C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\1344668698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Var_1[train.Var_1.isnull()] = 'Cat_6'
C:\Users\lovej\AppData\Local\Temp\ipykernel_9756\1344668698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Var_1[test.Var_1.isnull()] = 'Cat_6'


In [15]:
train.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [16]:
test.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

## 타겟 클래스 불균형 확인
불균형하지 않다고 판단

In [17]:
train.Segmentation.value_counts()

D    1388
A    1244
C    1143
B    1106
Name: Segmentation, dtype: int64

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4881 entries, 0 to 4880
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               4881 non-null   int64  
 1   Gender           4881 non-null   object 
 2   Ever_Married     4881 non-null   object 
 3   Age              4881 non-null   int64  
 4   Graduated        4881 non-null   object 
 5   Profession       4881 non-null   object 
 6   Work_Experience  4881 non-null   float64
 7   Spending_Score   4881 non-null   object 
 8   Family_Size      4881 non-null   float64
 9   Var_1            4881 non-null   object 
 10  Segmentation     4881 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 419.6+ KB


## Categorical 데이터 인코딩

In [19]:
label_columns = ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']
onehot_columns = ['Profession', 'Var_1'] 

In [22]:
train = pd.get_dummies(train, columns=onehot_columns, drop_first=True) 
test = pd.get_dummies(test, columns=onehot_columns, drop_first=True)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4881 entries, 0 to 4880
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        4881 non-null   int64  
 1   Gender                    4881 non-null   int32  
 2   Ever_Married              4881 non-null   int32  
 3   Age                       4881 non-null   int64  
 4   Graduated                 4881 non-null   int32  
 5   Work_Experience           4881 non-null   float64
 6   Spending_Score            4881 non-null   int32  
 7   Family_Size               4881 non-null   float64
 8   Segmentation              4881 non-null   object 
 9   Profession_Doctor         4881 non-null   uint8  
 10  Profession_Engineer       4881 non-null   uint8  
 11  Profession_Entertainment  4881 non-null   uint8  
 12  Profession_Executive      4881 non-null   uint8  
 13  Profession_Healthcare     4881 non-null   uint8  
 14  Professi

In [24]:
for label in label_columns:
    labelencoder = LabelEncoder()
    train[label] = labelencoder.fit_transform(train[label])
    test[label] = labelencoder.fit_transform(test[label])
    

In [25]:
# help(LabelEncoder)

## 모델링
랜덤포레스트 분류기와 XGBoost 분류기를 활용하여 macro f1_score 기준으로 가장 잘 분류하는 모델을 찾는다.

In [28]:
# train.drop(columns=onehot_columns, inplace=True)
# test.drop(columns=onehot_columns, inplace=True)

In [29]:
x = train.drop(columns=['ID', 'Segmentation'])
y = train.Segmentation 

In [30]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#help(train_test_split)

In [ ]:
help(RandomForestClassifier) 

In [31]:
rfc = RandomForestClassifier(random_state=42)

In [32]:
rfc.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [33]:
predict = rfc.predict(x_val)
f1_score(predict, y_val, average='macro')

0.39159051721380256

In [34]:
rfc1 = RandomForestClassifier(random_state=42, max_depth=8)

In [35]:
rfc1.fit(x_train, y_train)

RandomForestClassifier(max_depth=8, random_state=42)

In [36]:
predict1 = rfc1.predict(x_val)
f1_score(predict1, y_val, average='macro')

0.44561897933214456

In [37]:
xgb = XGBClassifier(random_state=42, max_depth=6)

In [38]:
le = LabelEncoder()
y_t = le.fit_transform(y_train)

In [39]:
le = LabelEncoder()
y_v = le.fit_transform(y_val)

In [40]:
xgb.fit(x_train, y_t)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [41]:
predict2 = xgb.predict(x_val)
f1_score(predict2, y_v, average='macro')

0.43736188085551975

## 결과 생성
최종 분류 모델로 max_depth 하이퍼파라미터를 지정한 랜덤포레스트 분류기를 사용한다.

In [44]:
rfc_final = RandomForestClassifier(random_state=42, max_depth=8)
rfc_final.fit(x, y)
predict_final = rfc_final.predict(test.drop(columns='ID'))

In [45]:
submission = pd.DataFrame()
submission['ID'] = test['ID']
submission['Segmentation'] = predict_final
submission

,ID,Segmentation
0,460406,D
1,466890,C
2,466145,C
3,465805,B
4,466137,D
...,...,...
1537,463734,B
1538,466055,B
1539,465689,D
1540,460401,D


In [46]:
submission.to_csv('000000.csv', index=False)